In [4]:
import logging
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.common.exceptions import WebDriverException

from lavague.core import WorldModel, ActionEngine
from lavague.core.agents import WebAgent
from lavague.drivers.selenium import SeleniumDriver
from lavague.contexts.openai import AzureOpenaiContext
# Configure logging
logging.basicConfig(level=logging.DEBUG)

In [5]:
def main():
    # Set up Chrome options
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Run in headless mode
    chrome_options.add_argument("--disable-gpu")  # Disable GPU acceleration
    chrome_options.add_argument("--no-sandbox")  # Add no-sandbox argument
    chrome_options.add_argument(
        "--disable-dev-shm-usage"
    )  # Add disable-dev-shm-usage argument

    # Set up the remote WebDriver
    browserless_url = "http://localhost:3000/webdriver"

    try:
        driver = webdriver.Remote(
            command_executor=browserless_url, options=chrome_options
        )
        return driver

    except WebDriverException as e:
        logging.error(f"WebDriverException: {e}")
    except Exception as e:
        logging.error(f"Exception: {e}")
    finally:
        if "driver" in locals():
            driver.quit()


if __name__ == "__main__":
    driver = main()
    context = AzureOpenaiContext()
    context.mm_llm = AzureOpenaiContext().mm_llm
    selenium_driver = SeleniumDriver(headless=False)
    world_model = WorldModel.from_context(context)
    action_engine = ActionEngine.from_context(context, selenium_driver)
    agent = WebAgent(world_model, action_engine)
    agent.get("https://huggingface.co/docs")
    agent.run("Go on the quicktour of PEFT")

DEBUG:selenium.webdriver.remote.remote_connection:POST http://localhost:3000/webdriver/session {'capabilities': {'firstMatch': [{}], 'alwaysMatch': {'browserName': 'chrome', 'pageLoadStrategy': <PageLoadStrategy.normal: 'normal'>, 'goog:chromeOptions': {'extensions': [], 'args': ['--headless', '--disable-gpu', '--no-sandbox', '--disable-dev-shm-usage']}}}}
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): localhost:3000
DEBUG:urllib3.connectionpool:http://localhost:3000 "POST /webdriver/session HTTP/1.1" 200 0
DEBUG:selenium.webdriver.remote.remote_connection:Remote response: status=200 | data={"value":{"capabilities":{"acceptInsecureCerts":false,"browserName":"chrome-headless-shell","browserVersion":"121.0.6167.85","chrome":{"chromedriverVersion":"121.0.6167.85 (3f98d690ad7e59242ef110144c757b2ac4eef1a2-refs/branch-heads/6167@{#1539})","userDataDir":"/tmp/browserless-data-dir-Jr32pX"},"fedcm:accounts":true,"goog:chromeOptions":{"debuggerAddress":"localhost:42385"},"network

ValidationError: 1 validation error for AzureOpenAI
model
  none is not an allowed value (type=type_error.none.not_allowed)